# Distinguish supported and unsupported rearing

1. Determine the board line for support and unsupport
2. Detect the rearing
3. Analyze and find the difference between supporte and unsupported


Have not finished yet

# 1. Load the data

# 

In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from glob import glob
from base import MultiDaysBeaconPosition, BeaconPosition
from scipy.stats import sem
from utils.basic_utils import get_tags, joint_density_function
from sklearn.decomposition import PCA
from utils.pca_method import generate_tau_sample
from utils.baseline_method import continuous_detection 

In [3]:
root_path = '../Data/Raw/'
rat_id = 'FS10'
rat_subdirectories = glob(root_path+rat_id+'/*/' )

tags = get_tags(rat_subdirectories)

## 1.1 For one tag of rat

In [4]:
tag_id = 0
this_tag = BeaconPosition(rat_subdirectories[tag_id], tags[tag_id], True, True) #position data of this tag

In [5]:
unit_time = np.mean(this_tag.position_data[1:,0] - this_tag.position_data[:-1,0])
print(unit_time) # average time for a bin

0.016694419627324594


# 2. Detect rearing roughly (using threshold first)

In [10]:
# set the parameters
save_fig = False # whether save the fig for the results

T = 60
tau = 20
group_step = 10 # so that no overlap between taus

height_thresh = 0.6 # threshold for height/z to determine rearing

# get some important variables
xy_speed = np.insert(this_tag.speed,0,0) # speed on xy planar insert 0 at the beginning of the speed array to make the size same
z_speed = np.divide(this_tag.position_data[1:,3] - this_tag.position_data[:-1,3], this_tag.position_data[1:,0] - this_tag.position_data[:-1,0])
z_speed = np.insert(z_speed,0,0) # insert 0 at the beginning of the speed array to make the size same 
x_posi = this_tag.position_data[:,1]
y_posi = this_tag.position_data[:,2]
z = this_tag.position_data[:,3]

# organize k features that we're interested in
k = 6
all_feature_matrix = np.zeros((k, len(z)))

# normalize each feature, so that the scale of max value is 1
all_feature_matrix[0,:] = z
all_feature_matrix[1,:] = this_tag.position_data[:,6]
all_feature_matrix[2,:] = z_speed
all_feature_matrix[3,:] = xy_speed
all_feature_matrix[4,:] = x_posi
all_feature_matrix[5,:] = y_posi


In [8]:
def rearing_detection(z, height_thresh = 0.6):
    '''
    detect rearing by the height_thresh, if z[i]> = height_thresh, then the time of z[i] if rearing time
    z: array of height in time series
    height_thresh :threshold to determine the rearing
    
    return:
    r: array of index of rearing time
    '''
    
    return np.argwhere(z>= height_thresh).flatten()
    

In [16]:
rearing_idx = rearing_detection(z, height_thresh= height_thresh)
whether_continue, rearing_periods = continuous_detection(rearing_idx, discontinu_tolerance= 10, total_tolerance= 20,seperate= True)

# rearing_periods: slice of different rearing events

In [17]:
# for each rearing event, get T length time before it
T = 200
tau = 200
before_list = []
for i,this_rearing in enumerate(rearing_periods):
    begin_idx = this_rearing[0]
    if begin_idx - T >=0:
        before_list.append(all_feature_matrix[:,begin_idx - T: begin_idx])
before_matrix = np.asarray(before_list) # in shape [num_event, k, T]
num_events = before_matrix.shape[0]

In [19]:
# use a time window to slip along T and get slice of tau, combine different features horizontally, 
x_matrix = generate_tau_sample(before_matrix, tau =tau, step = tau, T = T,k=k)
x_norm = (x_matrix - x_matrix.mean(axis=0)) # global centering


# 3. Distribution of x, y

From Fabian code:
X_cut_min = -.6
Y_cut_max = 1.5
X_cut_max = .05
Y_cut_min = .08


In [25]:
print('max x = %.4f, min x = %.4f'%(max(x_posi), min(x_posi)))
print('max y = %.4f, min y = %.4f'%(max(y_posi), min(y_posi)))

max x = 0.1765, min x = -0.7113
max y = 1.7714, min y = -0.0211


In [27]:
# set the boarder 
# seems to be 0.1 different from the extreme value
x_min_boarder = -.6
y_max_boarder = 1.5
x_max_boarder = .05
y_min_boarder = .08